In [ ]:
#Author: Yiying Jiao

In [1]:
import os
import pandas as pd
import re

In [2]:

#this function takes in a path which leads to a csv file in my local directory
#strip the dataframe to only a date column and an url column, of unique rows that only mention
#apple(as a whole word)

def proccess_csv (path):
    df = pd.read_csv(path, on_bad_lines="skip", sep='\t')


    #--------extract relevanet columns------
    df = df.iloc[:, [1, (len(df.columns)-1)]]# Selects the 2nd and last columns

    # Step 1: Save the current column row (the mistaken column names)
    header_values = df.columns.tolist()  #get the current columns names(who are the first row)

    # Step 2: Create a DataFrame row from the column values
    header_df = pd.DataFrame([header_values], columns=df.columns)

    # Step 3: Concatenate the first row with the original DataFrame
    df_corrected = pd.concat([header_df, df], ignore_index=True)

    # Now assign new column names
    new_column_names = ["date", "url"]
    df_corrected.columns = new_column_names

    #rename for similicity
    df=df_corrected.copy()

    #----------filter---------
    #filter for mention of apple(as a whole word)
    filtered_df = df[df['url'].str.contains(r'\bapple\b', case=False, na=False, regex=True)]
    
    #filter to get rid of all duplicate links
    filtered_df = filtered_df.copy()
    filtered_df.drop_duplicates(subset='url', keep="first", inplace=True)
    
    #drop the index so it can be ready to concatenate into the big dataframe
    filtered_df = filtered_df.reset_index(drop=True)
    return filtered_df



In [ ]:

# Directory containing the CSV files
directory_path = "/Users/jiao/projects/csv_files_2023"

# Get all CSV files in the directory in a list
csv_file_list = [f for f in os.listdir(directory_path) if f.endswith(".CSV")]

main_df = pd.DataFrame()

# iterate through each csv, process each and save output
for file in csv_file_list:
    file_path = os.path.join(directory_path, file)  # Full path to the file
    # run the functionn, process the CSV
    curr_df = proccess_csv(file_path)
    urls = curr_df['url']
    curr_df['url'].fillna('', inplace=True)
    main_df = pd.concat([main_df, curr_df], ignore_index=True)


In [ ]:
print(main_df)

In [ ]:
main_df.to_csv("/Users/jiao/projects/23_main_df.csv", index=False)
